# Loading libraries and data

In [4]:
import pandas as pd
from datetime import datetime

In [5]:
df_train = pd.read_csv('data/event-log-training.csv')

In [6]:
df_train

eventID   case concept:name            case description case Class  \
0                0                  1  Simulated process instance      Print   
1                1                  1  Simulated process instance      Print   
2                2                  1  Simulated process instance      Print   
3                3                  1  Simulated process instance      Print   
4                4                  1  Simulated process instance      Print   
...            ...                ...                         ...        ...   
36623  42949673754                 18  Simulated process instance      Print   
36624  42949673755                 18  Simulated process instance      Print   
36625  42949673756                 18  Simulated process instance      Print   
36626  42949673757                 18  Simulated process instance      Print   
36627  42949673758                 18  Simulated process instance      Print   

      event org:resource         event concept:name  \
0                   PRN1                        Job   
1                   PRN1               Remote Print   
2                   PRN1         Read Print Options   
3                   PRN1              Rasterization   
4                   PRN1             Interpretation   
...                  ...                        ...   
36623               PRN1  Pressure Roller Spin Stop   
36624               PRN1                     Fusing   
36625               PRN1         Wipe Toner on Drum   
36626               PRN1       Erase Charge on Drum   
36627               PRN1                        Job   

      event lifecycle:transition     event time:timestamp  
0                          start  01-01-1970 01:00:00.000  
1                       complete  01-01-1970 01:15:00.000  
2                       complete  01-01-1970 01:26:00.000  
3                          start  01-01-1970 01:38:00.000  
4                          start  01-01-1970 01:51:00.000  
...                          ...                      ...  
36623                   complete  25-12-1970 15:51:00.000  
36624                   complete  25-12-1970 16:05:00.000  
36625                   complete  25-12-1970 16:10:00.000  
36626                   complete  25-12-1970 16:22:00.000  
36627                   complete  25-12-1970 16:44:00.000  

[36628 rows x 8 columns]

# Convert timestamp to datetime

In [7]:
date_list = []

for time in df_train['event time:timestamp']:
    datex = time[:-4]
    date = datetime.strptime(datex, '%d-%m-%Y %H:%M:%S')

    date_list.append(date)

In [8]:
df_train['time and date'] = date_list

In [9]:
df_train

eventID   case concept:name            case description case Class  \
0                0                  1  Simulated process instance      Print   
1                1                  1  Simulated process instance      Print   
2                2                  1  Simulated process instance      Print   
3                3                  1  Simulated process instance      Print   
4                4                  1  Simulated process instance      Print   
...            ...                ...                         ...        ...   
36623  42949673754                 18  Simulated process instance      Print   
36624  42949673755                 18  Simulated process instance      Print   
36625  42949673756                 18  Simulated process instance      Print   
36626  42949673757                 18  Simulated process instance      Print   
36627  42949673758                 18  Simulated process instance      Print   

      event org:resource         event concept:name  \
0                   PRN1                        Job   
1                   PRN1               Remote Print   
2                   PRN1         Read Print Options   
3                   PRN1              Rasterization   
4                   PRN1             Interpretation   
...                  ...                        ...   
36623               PRN1  Pressure Roller Spin Stop   
36624               PRN1                     Fusing   
36625               PRN1         Wipe Toner on Drum   
36626               PRN1       Erase Charge on Drum   
36627               PRN1                        Job   

      event lifecycle:transition     event time:timestamp       time and date  
0                          start  01-01-1970 01:00:00.000 1970-01-01 01:00:00  
1                       complete  01-01-1970 01:15:00.000 1970-01-01 01:15:00  
2                       complete  01-01-1970 01:26:00.000 1970-01-01 01:26:00  
3                          start  01-01-1970 01:38:00.000 1970-01-01 01:38:00  
4                          start  01-01-1970 01:51:00.000 1970-01-01 01:51:00  
...                          ...                      ...                 ...  
36623                   complete  25-12-1970 15:51:00.000 1970-12-25 15:51:00  
36624                   complete  25-12-1970 16:05:00.000 1970-12-25 16:05:00  
36625                   complete  25-12-1970 16:10:00.000 1970-12-25 16:10:00  
36626                   complete  25-12-1970 16:22:00.000 1970-12-25 16:22:00  
36627                   complete  25-12-1970 16:44:00.000 1970-12-25 16:44:00  

[36628 rows x 9 columns]

# Add in true next event and time it takes for case 4

In [10]:
df_train['event lifecycle:transition'].value_counts()

complete    33802
start        2826
Name: event lifecycle:transition, dtype: int64

In [11]:
dct_events = dict(df_train['event concept:name'].value_counts())

In [12]:
df_train['case concept:name'].value_counts()

12    2140
33    1594
42    1390
24    1285
6     1198
      ... 
27      62
80      59
56      55
4       43
51      43
Name: case concept:name, Length: 80, dtype: int64

In [13]:
df_train.groupby(by = ['case concept:name', 'time and date'], as_index = False)
df_case4 = df_train[df_train['case concept:name'] == 4]

In [14]:
#df_case4

Add event column

In [15]:
event_lst4 = [event for event in df_case4['event concept:name']]

event_lst4 = event_lst4[1:]
event_lst4.append('-')
    

In [16]:
df_case4['next_event'] = event_lst4

C:\Users\20204502\AppData\Local\Temp/ipykernel_16144/3880512715.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_case4['next_event'] = event_lst4


Add time to next event column

In [17]:
nexttime_lst1 = [time for time in df_case4['time and date']]

nexttime_lst = nexttime_lst1[1:]
nexttime_lst.append(nexttime_lst[-1])

In [18]:
time_diff = []

for i in range(len(nexttime_lst)):
    time_diff.append(nexttime_lst[i] - nexttime_lst1[i])

In [19]:
df_case4['time to next event'] = time_diff

C:\Users\20204502\AppData\Local\Temp/ipykernel_16144/1624469924.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_case4['time to next event'] = time_diff


In [20]:
df_case4.head()

eventID   case concept:name            case description case Class  \
64     146028888064                  4  Simulated process instance  Copy/Scan   
26983  146028888065                  4  Simulated process instance  Copy/Scan   
26984  146028888066                  4  Simulated process instance  Copy/Scan   
26985  146028888067                  4  Simulated process instance  Copy/Scan   
26986  146028888068                  4  Simulated process instance  Copy/Scan   

      event org:resource         event concept:name  \
64                  PRN1                        Job   
26983               PRN1                  Copy/Scan   
26984               PRN1  Collect Copy/Scan Options   
26985               PRN1                  Place Doc   
26986               PRN1        Illuminate Document   

      event lifecycle:transition     event time:timestamp       time and date  \
64                         start  01-01-1970 11:23:00.000 1970-01-01 11:23:00   
26983                   complete  29-09-1970 12:33:00.000 1970-09-29 12:33:00   
26984                   complete  29-09-1970 12:51:00.000 1970-09-29 12:51:00   
26985                   complete  29-09-1970 12:59:00.000 1970-09-29 12:59:00   
26986                   complete  29-09-1970 13:18:00.000 1970-09-29 13:18:00   

                      next_event time to next event  
64                     Copy/Scan  271 days 01:10:00  
26983  Collect Copy/Scan Options    0 days 00:18:00  
26984                  Place Doc    0 days 00:08:00  
26985        Illuminate Document    0 days 00:19:00  
26986             Move Scan Head    0 days 00:22:00

# Next event and time column for all data


In [21]:
cases = list(df_train['case concept:name'].unique())

In [19]:
df_result = pd.DataFrame()

In [29]:
int_dict = {}
for case in cases:
    df_case = df_train[df_train['case concept:name'] == case].reset_index()
    for index, row in df_case.iterrows():
        if row['event concept:name'] in int_dict:
            if index in int_dict[row['event concept:name']]:
                int_dict[row['event concept:name']][index] += 1
            else:
                int_dict[row['event concept:name']].update({index: 1})
        else:
            int_dict[row['event concept:name']] = {index: 1}

In [31]:
#2140
position_dict = {}

for i in range(2140):
    init = 0
    task = 0
    for key in int_dict.keys():
        try:
            new = int_dict[key][i]
        except:
            new = 0
        if new > init:
            init = new
            task = key

    position_dict.update({i: task})

In [32]:
position_dict

{0: 'Job',
 1: 'Remote Print',
 2: 'Read Print Options',
 3: 'Rasterization',
 4: 'Interpretation',
 5: 'Move Scan Head',
 6: 'Interpretation',
 7: 'A/D Conversion',
 8: 'Interpolation',
 9: 'Filtered Image',
 10: 'Collect Image',
 11: 'Illuminate Document',
 12: 'Move Scan Head',
 13: 'Focus Light Beam',
 14: 'A/D Conversion',
 15: 'Interpolation',
 16: 'Filtered Image',
 17: 'Illuminate Document',
 18: 'Accumulate Images',
 19: 'Move Scan Head',
 20: 'Focus Light Beam',
 21: 'A/D Conversion',
 22: 'Interpolation',
 23: 'Filtered Image',
 24: 'Collect Image',
 25: 'Illuminate Document',
 26: 'Move Scan Head',
 27: 'Focus Light Beam',
 28: 'A/D Conversion',
 29: 'Interpolation',
 30: 'Filtered Image',
 31: 'Illuminate Document',
 32: 'Collect Image',
 33: 'Move Scan Head',
 34: 'Focus Light Beam',
 35: 'A/D Conversion',
 36: 'Interpolation',
 37: 'Filtered Image',
 38: 'Illuminate Document',
 39: 'Collect Image',
 40: 'Move Scan Head',
 41: 'Focus Light Beam',
 42: 'A/D Conversion',
 4

In [ ]:
#create next time list
    nexttime_lst1 = [time for time in df_case['time and date']]
    nexttime_lst = nexttime_lst1[1:]
    nexttime_lst.append(nexttime_lst[-1])
    
    #create difference list
    time_diff = []
    for i in range(len(nexttime_lst)):
        time_diff.append(nexttime_lst[i] - nexttime_lst1[i])

In [38]:
df_result = pd.DataFrame()

for case in cases: 
    
    df_case = df_train[df_train['case concept:name'] == case].copy().reset_index()

    pred_list = [position_dict[i] for i in range(len(df_case))]
    pred_list = pred_list[1:]
    pred_list.append('-')

    

    
    #create next event list
    event_lst = [event for event in df_case['event concept:name']]
    event_lst = event_lst[1:]
    event_lst.append('-')
    
    #create next time list
    nexttime_lst1 = [time for time in df_case['time and date']]
    nexttime_lst = nexttime_lst1[1:]
    nexttime_lst.append(nexttime_lst[-1])
    
    #create difference list
    time_diff = []
    for i in range(len(nexttime_lst)):
        time_diff.append(nexttime_lst[i] - nexttime_lst1[i])
    
    #append columns to df case
    df_case['Next event'] = event_lst
    df_case['Time to next event'] = time_diff


    df_case['Event predcition'] = pred_list
    
    #append df case to final df
    df_result = df_result.append(df_case)

In [41]:
df_result.head()

index  eventID   case concept:name            case description case Class  \
0      0         0                  1  Simulated process instance      Print   
1      1         1                  1  Simulated process instance      Print   
2      2         2                  1  Simulated process instance      Print   
3      3         3                  1  Simulated process instance      Print   
4      4         4                  1  Simulated process instance      Print   

  event org:resource  event concept:name event lifecycle:transition  \
0               PRN1                 Job                      start   
1               PRN1        Remote Print                   complete   
2               PRN1  Read Print Options                   complete   
3               PRN1       Rasterization                      start   
4               PRN1      Interpretation                      start   

      event time:timestamp       time and date          Next event  \
0  01-01-1970 01:00:00.000 1970-01-01 01:00:00        Remote Print   
1  01-01-1970 01:15:00.000 1970-01-01 01:15:00  Read Print Options   
2  01-01-1970 01:26:00.000 1970-01-01 01:26:00       Rasterization   
3  01-01-1970 01:38:00.000 1970-01-01 01:38:00      Interpretation   
4  01-01-1970 01:51:00.000 1970-01-01 01:51:00    Unformatted Text   

  Time to next event    Event predcition  
0    0 days 00:15:00        Remote Print  
1    0 days 00:11:00  Read Print Options  
2    0 days 00:12:00       Rasterization  
3    0 days 00:13:00      Interpretation  
4    0 days 00:18:00      Move Scan Head

In [42]:
time_dict = {}
for case in cases:
    df_case = df_result[df_result['case concept:name'] == case].copy()
    df_case.reset_index(inplace=True)
    for index, row in df_case.iterrows():
        if index in time_dict:
            time_dict[index]['sum'] += row['Time to next event']
            time_dict[index]['count'] += 1
        else:
            time_dict[index] = {'sum': row['Time to next event'], 'count': 1}
    

    

In [45]:
mean_time_dict = {}
for position in time_dict.keys():
    mean_time = time_dict[position]['sum'] / time_dict[position]['count']
    mean_time_dict[position] = mean_time

In [49]:
mean_time_dict

{0: Timedelta('164 days 09:23:36.750000'),
 1: Timedelta('0 days 00:14:24'),
 2: Timedelta('0 days 00:15:29.250000'),
 3: Timedelta('0 days 00:15:31.500000'),
 4: Timedelta('0 days 00:15:10.500000'),
 5: Timedelta('0 days 00:15:08.250000'),
 6: Timedelta('0 days 00:14:06.750000'),
 7: Timedelta('0 days 00:10:15.750000'),
 8: Timedelta('0 days 00:10:15'),
 9: Timedelta('0 days 00:11:44.250000'),
 10: Timedelta('0 days 00:04:41.250000'),
 11: Timedelta('0 days 00:12:03'),
 12: Timedelta('0 days 00:11:21.750000'),
 13: Timedelta('0 days 00:10:21'),
 14: Timedelta('0 days 00:10:30.750000'),
 15: Timedelta('0 days 00:10:29.250000'),
 16: Timedelta('0 days 00:09:24'),
 17: Timedelta('0 days 00:04:30.750000'),
 18: Timedelta('0 days 00:11:17.250000'),
 19: Timedelta('0 days 00:10:30'),
 20: Timedelta('0 days 00:11:03'),
 21: Timedelta('0 days 00:12:23.250000'),
 22: Timedelta('0 days 00:11:48'),
 23: Timedelta('0 days 00:11:29.250000'),
 24: Timedelta('0 days 00:03:54'),
 25: Timedelta('0 day

In [50]:
lst = []
for case in cases:
    df_case = df_result[df_result['case concept:name'] == case].copy()
    df_case.reset_index(inplace=True)
    for index, row in df_case.iterrows():
        time = mean_time_dict[index]
        lst.append(time)

        



In [52]:
df_result['Time prediction'] = lst

In [53]:
df_result

index    eventID   case concept:name            case description  \
0       0           0                  1  Simulated process instance   
1       1           1                  1  Simulated process instance   
2       2           2                  1  Simulated process instance   
3       3           3                  1  Simulated process instance   
4       4           4                  1  Simulated process instance   
..    ...         ...                ...                         ...   
73   9579  8589934665                100  Simulated process instance   
74   9580  8589934666                100  Simulated process instance   
75   9581  8589934667                100  Simulated process instance   
76   9582  8589934668                100  Simulated process instance   
77   9583  8589934669                100  Simulated process instance   

   case Class event org:resource  event concept:name  \
0       Print               PRN1                 Job   
1       Print               PRN1        Remote Print   
2       Print               PRN1  Read Print Options   
3       Print               PRN1       Rasterization   
4       Print               PRN1      Interpretation   
..        ...                ...                 ...   
73  Copy/Scan               PRN1         Compression   
74  Copy/Scan               PRN1         Store Image   
75  Copy/Scan               PRN1      Transfer Image   
76  Copy/Scan               PRN1           Send SMTP   
77  Copy/Scan               PRN1                 Job   

   event lifecycle:transition     event time:timestamp       time and date  \
0                       start  01-01-1970 01:00:00.000 1970-01-01 01:00:00   
1                    complete  01-01-1970 01:15:00.000 1970-01-01 01:15:00   
2                    complete  01-01-1970 01:26:00.000 1970-01-01 01:26:00   
3                       start  01-01-1970 01:38:00.000 1970-01-01 01:38:00   
4                       start  01-01-1970 01:51:00.000 1970-01-01 01:51:00   
..                        ...                      ...                 ...   
73                   complete  29-03-1970 16:25:00.000 1970-03-29 16:25:00   
74                   complete  29-03-1970 16:49:00.000 1970-03-29 16:49:00   
75                   complete  29-03-1970 17:06:00.000 1970-03-29 17:06:00   
76                   complete  29-03-1970 17:22:00.000 1970-03-29 17:22:00   
77                   complete  29-03-1970 17:33:00.000 1970-03-29 17:33:00   

            Next event Time to next event    Event predcition  \
0         Remote Print    0 days 00:15:00        Remote Print   
1   Read Print Options    0 days 00:11:00  Read Print Options   
2        Rasterization    0 days 00:12:00       Rasterization   
3       Interpretation    0 days 00:13:00      Interpretation   
4     Unformatted Text    0 days 00:18:00      Move Scan Head   
..                 ...                ...                 ...   
73         Store Image    0 days 00:24:00           Screening   
74      Transfer Image    0 days 00:17:00      Move Scan Head   
75           Send SMTP    0 days 00:16:00           Screening   
76                 Job    0 days 00:11:00      A/D Conversion   
77                   -    0 days 00:00:00                   -   

             Time prediction  
0   164 days 09:23:36.750000  
1            0 days 00:14:24  
2     0 days 00:15:29.250000  
3     0 days 00:15:31.500000  
4     0 days 00:15:10.500000  
..                       ...  
73           0 days 00:09:05  
74           0 days 00:13:50  
75 0 days 00:12:31.666666666  
76 0 days 00:10:55.833333333  
77 0 days 00:13:36.338028169  

[36628 rows x 14 columns]

In [ ]:
int_dict = {}
for case in cases:
    df_case = df_train[df_train['case concept:name'] == case].reset_index()
    for index, row in df_case.iterrows():
        if row['event concept:name'] in int_dict:
            if index in int_dict[row['event concept:name']]:
                int_dict[row['event concept:name']][index] += 1
            else:
                int_dict[row['event concept:name']].update({index: 1})
        else:
            int_dict[row['event concept:name']] = {index: 1}

In [1]:
df_result.head()

NameError: name 'df_result' is not defined